In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
fcc_df = pd.read_csv('FCC_477_All.csv') 

In [3]:
fcc_df=fcc_df[fcc_df['Technology Code'] != 60] 
fcc_df=fcc_df[fcc_df['Max Advertised Downstream Speed (mbps)'] >= 25] 
fcc_df=fcc_df[fcc_df['Max Advertised Upstream Speed (mbps)'] >= 3] 
fcc_df=fcc_df[fcc_df['Consumer'] == 1]

In [4]:
fcc_df = fcc_df.drop(['Holding Company Number','Consumer'],axis=1)

In [5]:
#fcc_df.to_csv('fcc_filtered_all.csv')

In [6]:
fcc_df.head()

,Logical Record Number,Provider ID,FRN,Provider Name,DBA Name,Holding Company Name,Holding Company Final,State,Census Block FIPS Code,Technology Code,Max Advertised Downstream Speed (mbps),Max Advertised Upstream Speed (mbps),Business,Max CIR Downstream Speed (mbps),Max CIR Upstream Speed (mbps)
14378296,1,43424,27178078,Element Wireless,Element Wireless,Element Wireless,Element Wireless,MO,290239501004122,70,30.0,10.0,0,0.0,0.0
14378297,2,43424,27178078,Element Wireless,Element Wireless,Element Wireless,Element Wireless,MO,290239501004120,70,45.0,15.0,1,45.0,15.0
14378299,4,43424,27178078,Element Wireless,Element Wireless,Element Wireless,Element Wireless,MO,292074702001116,70,30.0,10.0,0,0.0,0.0
14378300,5,43424,27178078,Element Wireless,Element Wireless,Element Wireless,Element Wireless,MO,290239501004150,70,30.0,10.0,0,0.0,0.0
14378302,7,43424,27178078,Element Wireless,Element Wireless,Element Wireless,Element Wireless,MO,292074702001268,70,30.0,10.0,0,0.0,0.0


In [7]:
 fp = "tl_2017_47_tabblock10.shp"

In [8]:
data = gpd.read_file(fp)

In [9]:
type(data)

geopandas.geodataframe.GeoDataFrame

In [10]:
data["GEOID10"] = pd.to_numeric(data["GEOID10"])

In [11]:
all_geo = pd.merge(data,fcc_df,left_on='GEOID10',right_on='Census Block FIPS Code')

In [12]:
all_geo.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYPE,...,Holding Company Name,Holding Company Final,State,Census Block FIPS Code,Technology Code,Max Advertised Downstream Speed (mbps),Max Advertised Upstream Speed (mbps),Business,Max CIR Downstream Speed (mbps),Max CIR Upstream Speed (mbps)
0,47,115,050201,3033,471150502013033,Block 3033,G5040,R,None,None,...,Charter Communications,Charter Communications,TN,471150502013033,43,940.0,35.0,1,0.0,0.0
1,47,115,050201,3095,471150502013095,Block 3095,G5040,R,None,None,...,Charter Communications,Charter Communications,TN,471150502013095,43,940.0,35.0,1,0.0,0.0
2,47,115,050302,1081,471150503021081,Block 1081,G5040,R,None,None,...,Charter Communications,Charter Communications,TN,471150503021081,43,940.0,35.0,1,0.0,0.0
3,47,043,060501,4016,470430605014016,Block 4016,G5040,R,None,None,...,Comcast Corporation,Comcast Corporation,TN,470430605014016,43,987.0,35.0,1,0.0,0.0
4,47,043,060501,4016,470430605014016,Block 4016,G5040,R,None,None,...,AT&T Inc.,AT&T Inc.,TN,470430605014016,12,25.0,5.0,0,0.0,0.0


In [13]:
us_block = 'us_block.shp'

In [14]:
us_df = gpd.read_file(us_block)

In [15]:
us_df.shape

(219831, 54)

In [16]:
pd.set_option('display.max_columns', None)
us_df.head()

,OBJECTID,STATE_FIPS,CNTY_FIPS,STCOFIPS,TRACT,BLKGRP,FIPS,POPULATION,POP_SQMI,POP2010,POP10_SQMI,WHITE,BLACK,AMERI_ES,ASIAN,HAWN_PI,HISPANIC,OTHER,MULT_RACE,MALES,FEMALES,AGE_UNDER5,AGE_5_9,AGE_10_14,AGE_15_19,AGE_20_24,AGE_25_34,AGE_35_44,AGE_45_54,AGE_55_64,AGE_65_74,AGE_75_84,AGE_85_UP,MED_AGE,MED_AGE_M,MED_AGE_F,HOUSEHOLDS,AVE_HH_SZ,HSEHLD_1_M,HSEHLD_1_F,MARHH_CHD,MARHH_NO_C,MHH_CHILD,FHH_CHILD,FAMILIES,AVE_FAM_SZ,HSE_UNITS,VACANT,OWNER_OCC,RENTER_OCC,SQMI,Shape_Leng,Shape_Area,geometry
0,1.0,15,003,15003,981200,1,150039812001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,3.10,0.459320,0.000724,"MULTIPOLYGON (((-161.93000 23.06562, -161.9297..."
1,2.0,15,009,15009,031801,2,150090318012,1705.0,40.0,1620.0,38.0,156.0,6.0,1.0,269.0,440.0,109.0,6.0,742.0,824.0,796.0,126.0,134.0,136.0,143.0,98.0,178.0,188.0,213.0,191.0,130.0,52.0,31.0,34.7,32.6,36.6,472.0,3.39,38.0,39.0,129.0,128.0,22.0,52.0,371.0,3.82,542.0,70.0,347.0,125.0,42.62,1.023727,0.009600,"POLYGON ((-157.01409 21.18396, -157.01402 21.1..."
2,3.0,15,003,15003,000500,3,150030005003,931.0,4433.3,917.0,4366.7,371.0,1.0,1.0,364.0,25.0,20.0,2.0,153.0,443.0,474.0,34.0,56.0,62.0,57.0,29.0,98.0,96.0,139.0,169.0,77.0,61.0,39.0,47.0,45.6,48.3,322.0,2.85,17.0,32.0,90.0,126.0,2.0,10.0,248.0,3.10,385.0,63.0,253.0,69.0,0.21,0.039725,0.000048,"POLYGON ((-157.78349 21.26293, -157.78370 21.2..."
3,4.0,15,003,15003,000500,5,150030005005,1009.0,4387.0,972.0,4226.1,359.0,5.0,1.0,446.0,24.0,19.0,0.0,137.0,480.0,492.0,47.0,55.0,69.0,54.0,34.0,78.0,126.0,173.0,160.0,97.0,54.0,25.0,46.6,46.5,46.8,346.0,2.81,22.0,32.0,108.0,136.0,8.0,6.0,275.0,3.10,396.0,50.0,263.0,83.0,0.23,0.029951,0.000052,"POLYGON ((-157.78600 21.27242, -157.78594 21.2..."
4,5.0,15,003,15003,000600,1,150030006001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1.0,0.0,0.0,0.81,0.058842,0.000183,"POLYGON ((-157.79905 21.26782, -157.79908 21.2..."


In [17]:
pd.set_option('display.max_columns', None)
fcc_df.head()

,Logical Record Number,Provider ID,FRN,Provider Name,DBA Name,Holding Company Name,Holding Company Final,State,Census Block FIPS Code,Technology Code,Max Advertised Downstream Speed (mbps),Max Advertised Upstream Speed (mbps),Business,Max CIR Downstream Speed (mbps),Max CIR Upstream Speed (mbps)
14378296,1,43424,27178078,Element Wireless,Element Wireless,Element Wireless,Element Wireless,MO,290239501004122,70,30.0,10.0,0,0.0,0.0
14378297,2,43424,27178078,Element Wireless,Element Wireless,Element Wireless,Element Wireless,MO,290239501004120,70,45.0,15.0,1,45.0,15.0
14378299,4,43424,27178078,Element Wireless,Element Wireless,Element Wireless,Element Wireless,MO,292074702001116,70,30.0,10.0,0,0.0,0.0
14378300,5,43424,27178078,Element Wireless,Element Wireless,Element Wireless,Element Wireless,MO,290239501004150,70,30.0,10.0,0,0.0,0.0
14378302,7,43424,27178078,Element Wireless,Element Wireless,Element Wireless,Element Wireless,MO,292074702001268,70,30.0,10.0,0,0.0,0.0


In [18]:
fcc_df = fcc_df.drop(['Logical Record Number','FRN','DBA Name','Holding Company Name','Holding Company Final','Business','Max CIR Downstream Speed (mbps)','Max CIR Upstream Speed (mbps)'],axis=1)

In [19]:
fcc_df['fips'] = fcc_df['Census Block FIPS Code'].astype(str).str[:13]
fcc_df.head()

,Provider ID,Provider Name,State,Census Block FIPS Code,Technology Code,Max Advertised Downstream Speed (mbps),Max Advertised Upstream Speed (mbps),fips
14378296,43424,Element Wireless,MO,290239501004122,70,30.0,10.0,2902395010041
14378297,43424,Element Wireless,MO,290239501004120,70,45.0,15.0,2902395010041
14378299,43424,Element Wireless,MO,292074702001116,70,30.0,10.0,2920747020011
14378300,43424,Element Wireless,MO,290239501004150,70,30.0,10.0,2902395010041
14378302,43424,Element Wireless,MO,292074702001268,70,30.0,10.0,2920747020012


In [20]:
fcc_df = fcc_df.drop(['Census Block FIPS Code'],axis=1)


In [21]:
fcc_df['FIPS'] = fcc_df['fips']


In [22]:
cols = [0,1,2,4,5]
us_df.drop(us_df.columns[cols],axis=1,inplace=True)

In [23]:
cols = [0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
us_df.drop(us_df.columns[cols],axis=1,inplace=True)

In [24]:
cols = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]
us_df.drop(us_df.columns[cols],axis=1,inplace=True)

In [25]:
us_df.head()


,FIPS,HOUSEHOLDS,Shape_Leng,Shape_Area,geometry
0,150039812001,0.0,0.459320,0.000724,"MULTIPOLYGON (((-161.93000 23.06562, -161.9297..."
1,150090318012,472.0,1.023727,0.009600,"POLYGON ((-157.01409 21.18396, -157.01402 21.1..."
2,150030005003,322.0,0.039725,0.000048,"POLYGON ((-157.78349 21.26293, -157.78370 21.2..."
3,150030005005,346.0,0.029951,0.000052,"POLYGON ((-157.78600 21.27242, -157.78594 21.2..."
4,150030006001,0.0,0.058842,0.000183,"POLYGON ((-157.79905 21.26782, -157.79908 21.2..."


In [26]:
fcc_df['FIPS'] = fcc_df['FIPS'].astype(str).str[:12]

In [27]:
fcc_df.head()

,Provider ID,Provider Name,State,Technology Code,Max Advertised Downstream Speed (mbps),Max Advertised Upstream Speed (mbps),fips,FIPS
14378296,43424,Element Wireless,MO,70,30.0,10.0,2902395010041,290239501004
14378297,43424,Element Wireless,MO,70,45.0,15.0,2902395010041,290239501004
14378299,43424,Element Wireless,MO,70,30.0,10.0,2920747020011,292074702001
14378300,43424,Element Wireless,MO,70,30.0,10.0,2902395010041,290239501004
14378302,43424,Element Wireless,MO,70,30.0,10.0,2920747020012,292074702001


In [28]:
merge_df = pd.merge(fcc_df,us_df,on='FIPS')

In [29]:
merge_df.head()

,Provider ID,Provider Name,State,Technology Code,Max Advertised Downstream Speed (mbps),Max Advertised Upstream Speed (mbps),fips,FIPS,HOUSEHOLDS,Shape_Leng,Shape_Area,geometry
0,43424,Element Wireless,MO,70,30.0,10.0,2902395010041,290239501004,488.0,0.667614,0.008831,"POLYGON ((-90.22102 36.84884, -90.22097 36.848..."
1,43424,Element Wireless,MO,70,45.0,15.0,2902395010041,290239501004,488.0,0.667614,0.008831,"POLYGON ((-90.22102 36.84884, -90.22097 36.848..."
2,43424,Element Wireless,MO,70,30.0,10.0,2902395010041,290239501004,488.0,0.667614,0.008831,"POLYGON ((-90.22102 36.84884, -90.22097 36.848..."
3,43424,Element Wireless,MO,70,30.0,10.0,2902395010041,290239501004,488.0,0.667614,0.008831,"POLYGON ((-90.22102 36.84884, -90.22097 36.848..."
4,43424,Element Wireless,MO,70,45.0,15.0,2902395010041,290239501004,488.0,0.667614,0.008831,"POLYGON ((-90.22102 36.84884, -90.22097 36.848..."


In [30]:
merge_df = merge_df.drop(['Provider Name','State','HOUSEHOLDS','Technology Code','Max Advertised Downstream Speed (mbps)','Max Advertised Upstream Speed (mbps)'],axis=1)

In [32]:
merge_df.head()

,Provider ID,fips,FIPS,Shape_Leng,Shape_Area,geometry
0,43424,2902395010041,290239501004,0.667614,0.008831,"POLYGON ((-90.22102 36.84884, -90.22097 36.848..."
1,43424,2902395010041,290239501004,0.667614,0.008831,"POLYGON ((-90.22102 36.84884, -90.22097 36.848..."
2,43424,2902395010041,290239501004,0.667614,0.008831,"POLYGON ((-90.22102 36.84884, -90.22097 36.848..."
3,43424,2902395010041,290239501004,0.667614,0.008831,"POLYGON ((-90.22102 36.84884, -90.22097 36.848..."
4,43424,2902395010041,290239501004,0.667614,0.008831,"POLYGON ((-90.22102 36.84884, -90.22097 36.848..."


In [33]:
merge_df.to_csv('fcc_all.csv')